In [13]:
import requests
import pandas as pd
import datetime
from datetime import datetime
import numpy as np
import sqlite3
import csv
import schedule
import time

### Purple Air's API

In [14]:
url_sensordata = "https://api.purpleair.com/v1/sensors/96317"

key_get = "C111944A-B3A7-11ED-B6F4-42010A800007"
key_post = "C112A231-B3A7-11ED-B6F4-42010A800007"
headers = {'X-API-Key': key_get}

In [15]:
# Get lat and lon from sensors.

#payload_lat = {'fields': 'latitude'}
#payload_lon = {'fields': 'longitude'}

#lat = requests.get(url_sensordata, headers = headers, params = payload_lat)
#lat = lat.json()
#lat = lat['sensor']['latitude']

#lon = requests.get(url_sensordata, headers = headers, params = payload_lon)
#lon = lon.json()
#lon = lon['sensor']['longitude']

### Create Sensor's database

In [16]:
# Connect to database.

conn = sqlite3.connect('aire-limpio.db')
c = conn.cursor()
c.execute("PRAGMA foreign_keys = ON;")

In [17]:
# Create the "sensors" table.

c.execute('''CREATE TABLE sensors
             (sensor_id INTEGER PRIMARY KEY, nombre TEXT, municipio TEXT, lat REAL, lon REAL)''')

conn.commit()

In [18]:
def insert_sensors_from_csv(file_path):
    
    # Connect to the SQLite database
    conn = sqlite3.connect('aire-limpio.db')
    c = conn.cursor()

    # Read the CSV file
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip the header row

        # Insert each row into the SQLite table
        for row in reader:
            sensor_id, nombre, municipio, lat, lon = row
            
            c.execute("INSERT INTO sensors (sensor_id, nombre, municipio, lat, lon) VALUES (?, ?, ?, ?, ?)",
                      (sensor_id, nombre, municipio, float(lat), float(lon)))

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

In [19]:
csv_file_path = 'assets/sensores.csv'
insert_sensors_from_csv(csv_file_path)

In [20]:
# Create the "aire_limpio" table.

c.execute('''CREATE TABLE aire_limpio
        ([pollution_id] INTEGER PRIMARY KEY, [sensor_id] INTEGER, [pm25] REAL, [pm10] REAL, [temp_celsius] REAL, [date] TEXT,
        FOREIGN KEY (sensor_id) REFERENCES sensors (sensor_id)) ''')

In [21]:
def fetch_sensor_data():
    
    # Connect to the SQLite database
    conn = sqlite3.connect('aire-limpio.db')
    c = conn.cursor()
    
    sensors = [143390, 50571, 132661, 143346, 144262, 133828, 144252, 93861, 144254, 93927, 33027, 34679, 34905, 36721, 38663, 42211, 43627, 50727, 50871, 50875, 56405, 104804,
    105602, 115335, 115885, 116099, 121719, 136522, 142670, 119965, 144260, 33117, 34499, 34511, 34569, 34641, 34969, 35011, 36709, 37253, 37255, 37643, 37657, 37661,
    37747, 37861, 39355, 39695, 39697, 39725, 39731, 39733, 45769, 49563, 50555, 50565, 118909, 121825, 140432, 140440, 143372, 147010, 173083, 25331, 175565, 35099,
    38475, 38537, 39285, 39509, 44463, 46511, 47771, 47855, 56439, 56441, 60085, 60227, 105714, 129911, 154481, 56657, 56429, 56643, 146946, 174777, 174731, 47883,
    93711, 39471, 119937, 132789, 133367, 47233, 174781, 36015, 174787, 45903, 93835, 24415, 143368, 96317]


    for sensor in sensors:

        url = f"https://api.purpleair.com/v1/sensors/{sensor}"

        sensor_id = sensor

        payload_all = {'fields': 'pm2.5, pm10.0, temperature'}

        response = requests.get(url, headers=headers, params=payload_all)
        data = response.json()

        if 'sensor' in data:
            sensor_data = data['sensor']

            pm25 = sensor_data.get('pm2.5', None)
            pm10 = sensor_data.get('pm10.0', None)
            temp = sensor_data.get('temperature', None)

            if temp is not None:
                temp_celsius = np.round((temp - 32) * .5556, decimals=0)
            else:
                temp_celsius = None

            date = datetime.now().strftime("%Y/%m/%d %H:%M:%S")

            c.execute('''INSERT INTO aire_limpio
                (sensor_id, pm25, pm10, temp_celsius, date) VALUES(?, ?, ?, ?, ?)''',
                (sensor_id, pm25, pm10, temp_celsius, date))

    conn.commit()
    conn.close()
    

In [22]:
def main():
    # Schedule the task to run every hour
    schedule.every(5).minutes.do(fetch_sensor_data)

    while True:
        # Run the pending tasks
        schedule.run_pending()
        time.sleep(60)  # Check for pending tasks every minute

In [23]:
if __name__ == "__main__":
    main()

KeyboardInterrupt: 

In [25]:
# Insert data into "aire_limpio" table.

sensors = [143390, 50571, 132661, 143346, 144262, 133828, 144252, 93861, 144254, 93927, 33027, 34679, 34905, 36721, 38663, 42211, 43627, 50727, 50871, 50875, 56405, 104804,
105602, 115335, 115885, 116099, 121719, 136522, 142670, 119965, 144260, 33117, 34499, 34511, 34569, 34641, 34969, 35011, 36709, 37253, 37255, 37643, 37657, 37661,
37747, 37861, 39355, 39695, 39697, 39725, 39731, 39733, 45769, 49563, 50555, 50565, 118909, 121825, 140432, 140440, 143372, 147010, 173083, 25331, 175565, 35099,
38475, 38537, 39285, 39509, 44463, 46511, 47771, 47855, 56439, 56441, 60085, 60227, 105714, 129911, 154481, 56657, 56429, 56643, 146946, 174777, 174731, 47883,
93711, 39471, 119937, 132789, 133367, 47233, 174781, 36015, 174787, 45903, 93835, 24415, 143368, 96317]


for sensor in sensors:
    
    url = f"https://api.purpleair.com/v1/sensors/{sensor}"
    
    sensor_id = sensor
    
    payload_all = {'fields': 'pm2.5, pm10.0, temperature'}

    response = requests.get(url, headers=headers, params=payload_all)
    data = response.json()

    if 'sensor' in data:
        sensor_data = data['sensor']

        pm25 = sensor_data.get('pm2.5', None)
        pm10 = sensor_data.get('pm10.0', None)
        temp = sensor_data.get('temperature', None)

        if temp is not None:
            temp_celsius = np.round((temp - 32) * .5556, decimals=0)
        else:
            temp_celsius = None

        date = datetime.now().strftime("%Y/%m/%d %H:%M:%S")

        c.execute('''INSERT INTO aire_limpio
            (sensor_id, pm25, pm10, temp_celsius, date) VALUES(?, ?, ?, ?, ?)''',
            (sensor_id, pm25, pm10, temp_celsius, date))
        
        conn.commit()

conn.close()

In [24]:
# Fetch data from aire_limpio
conn = sqlite3.connect('aire-limpio.db') 
c = conn.cursor()
                   
c.execute('''
          SELECT
          pollution_id, sensor_id, pm25, pm10, temp_celsius, date FROM aire_limpio;
          ''')

df = pd.DataFrame(c.fetchall(), columns=['pollution_id', 'sensor_id', 'pm25', 'pm10', 'temp_celsius', 'date'])
df

,pollution_id,sensor_id,pm25,pm10,temp_celsius,date
0,1,143390,5.5,7.5,26.0,2023/05/02 00:34:48
1,2,50571,6.0,6.6,22.0,2023/05/02 00:34:49
2,3,132661,4.5,4.9,26.0,2023/05/02 00:34:49
3,4,143346,10.6,12.0,25.0,2023/05/02 00:34:49
4,5,144262,7.2,10.4,25.0,2023/05/02 00:34:49
...,...,...,...,...,...,...
709,710,45903,10.9,12.2,27.0,2023/05/02 00:52:55
710,711,93835,19.9,22.5,24.0,2023/05/02 00:52:55
711,712,24415,2.2,2.9,30.0,2023/05/02 00:52:56
712,713,143368,17.5,18.8,36.0,2023/05/02 00:52:56


In [25]:
# Fetch data from sensors
conn = sqlite3.connect('aire-limpio.db') 
c = conn.cursor()
                   
c.execute('''
          SELECT
          sensor_id, municipio, lat, lon FROM sensors;
          ''')

df = pd.DataFrame(c.fetchall(), columns=['sensor_id', 'municipio', 'lat', 'lon'])
df

,sensor_id,municipio,lat,lon
0,24415,San Nicolas de los Garza,25.727991,-100.227165
1,25331,San Pedro Garza Garcia,25.644740,-100.403950
2,33027,Monterrey,25.600061,-100.262560
3,33117,San Pedro Garza Garcia,25.650322,-100.328490
4,34499,San Pedro Garza Garcia,25.641743,-100.347200
...,...,...,...,...
97,174731,Monterrey,25.745853,-100.412834
98,174777,Garcia,25.763336,-100.457360
99,174781,Monterrey,25.696613,-100.348114
100,174787,Escobedo,25.779789,-100.280210


temp_celsius